In [5]:
import geopandas
import matplotlib.pyplot as plt
import overpass
import numpy as np
import pandas as pd
import time

In [2]:
def make_feature_dict(df, features, prefix=None):
    """
    
    Parameters
    ----------
    df : geopandas.geodataframe.GeoDataFrame
        Dataframe containing features pulled from overpass API
        
    features : list
        List of column names associated with high-level feature names
        employed by overpass/OSM. Reference here:
        https://wiki.openstreetmap.org/wiki/Map_Features
        
    prefix : str
        Additional identifier placed before feature name.
        
    Returns
    -------
    dict
        Nested dictionary of features with value counts as eventual values.
        
    """
    
    if prefix!=None:
        pass
    else:
        prefix=''
        
    feature_dict = {}
    for feature in features:
        if feature in df.columns:
            series = df[feature].value_counts()
            feature_dict[prefix+feature] = { k:v for (k,v) in zip(series.index, series.values)}
        else:
            feature_dict[prefix+feature] = None
    return feature_dict

In [3]:
collision_per_aadt = geopandas.read_file('pa_collsion_per_traffic_191002.shp')

In [7]:
cols = [
    'aeroway',
    'amenity',
    'barrier',
    'boundary',
    'building',
    'healthcare',
    'highway',
    'landuse',
    'leisure',
    'man_made',
    'natural',
    'parking',
    'power',
    'railway',
    'route',
    'service',
    'surface',
    'tourism',
    'waterway'
]

In [ ]:
recollect

In [17]:
'overpass_one_mile_191002/crashes/{}.csv'.format(str(i).rjust(5,'0'))

'overpass_one_mile_191002/crashes/10164.csv'

In [18]:
endpoint = 'https://overpass.kumi.systems/api/interpreter'
endpoint = 'https://z.overpass-api.de/api/interpreter'
endpoint = 'https://lz4.overpass-api.de/api/interpreter'
api_pass = overpass.API(timeout=7200, endpoint=endpoint)
road_responses, area_responses = list(), list()
i = 10201

for index, row in collision_per_aadt.iloc[i:,:].iterrows(): 
    print(i, end=', ')
    poly = row.geometry.buffer(.00375, cap_style=3)
    test_shape = poly.simplify(0.001, preserve_topology=False)
    
    # Building overpass queries
    road_query = 'nwr(around:1, {c[1]}, {c[0]});(._;>;);out;'.format(
        c=row.geometry.interpolate(0.5, normalized = True).coords[:][0])
    area_query = 'nwr(poly:"{}");out;'.format(
        ' '.join('{c[1]} {c[0]}'.format(c=c) for c in test_shape.exterior.coords[:]))
    
    # Getting overpass road responses
    road_response = api_pass.get(road_query)
    #road_responses.append(road_response)
    road_geodf = geopandas.GeoDataFrame.from_features(road_response)
    culled_road = road_geodf.dropna(axis='index', how='all', subset=road_geodf.columns[1:])
    # Overwriting feature_dict each time
    feature_dict = make_feature_dict(culled_road, ['highway', 'surface'], prefix='road_')
    time.sleep(1)
    
    # Getting overpass area responses
    response = api_pass.get(area_query)
    #area_responses.append(response)
    geodf = geopandas.GeoDataFrame.from_features(response)
    culled_geodf = geodf.dropna(axis='index', how='all', subset=geodf.columns[1:])
    feature_dict.update(make_feature_dict(culled_geodf, cols, prefix='area_'))
    pd.io.json.json_normalize(feature_dict).to_csv('overpass_one_mile_191002/crashes/{}.csv'.format(str(i).rjust(4,'0')))
    time.sleep(1)
    i += 1

10164, 10165, 10166, 10167, 10168, 10169, 10170, 10171, 10172, 10173, 10174, 10175, 10176, 10177, 10178, 10179, 10180, 10181, 10182, 10183, 10184, 10185, 10186, 10187, 10188, 10189, 10190, 10191, 10192, 10193, 10194, 10195, 10196, 10197, 10198, 10199, 10200, 10201, 

KeyboardInterrupt: 